In [ ]:
from spacecraft.satellite import SatelliteImplementation
from setup.initial_settings import SimulationSetupReader
from setup.two_line_element import TwoLineElementReader
from visualziations.visualizations import plot_orbit, plot_lla, plot_position, plot_magnetic_field_eci, plot_magnetic_field_sbf, plot_sun_vector_eci, plot_sun_vector_sbf
from visualziations.visualizations import plot_angular_velocity, plot_euler_angles, plot_pointing_error, plot_torque, plot_angular_acceleration
from spacecraft.sensors import MagnetometerImplementation, SunsensorImplementation, SensorFusionImplementation
import core.transformations as tr
import core.utilities as ut
from pathlib import Path

import numpy as np

setup = SimulationSetupReader(Path('setup/initial_settings.json'))
tle = TwoLineElementReader(Path('setup/tle'))
magnetometer = MagnetometerImplementation(setup)
sunsensor = SunsensorImplementation(setup)
sensor_fusion = SensorFusionImplementation(setup, ['triad', 'quest', 'ekf'], tr.euler_xyz_to_quaternion(setup.euler_angles))

satellite = SatelliteImplementation(setup, tle, magnetometer, sunsensor, sensor_fusion)
quaternion_prev = satellite.quaternion.copy()
satellite.state_vector.next_row()
ut.basic_state_vector(satellite)

n_iter = setup.iterations_info["stop"]
for x in range(setup.iterations_info["start"], n_iter, setup.iterations_info["step"]):
    satellite.update_iteration(x)
    satellite.apply_rotation()

    mag_field_sbf, mag_field_eci = satellite.magnetic_field
    sun_vector_sbf, sun_vector_eci = satellite.sun_vector

    if x % setup.iterations_info["log_interval"] == 0:
        print(f"Iteration {x} of {n_iter}")

    satellite.fuse_sensors(
        [mag_field_sbf, sun_vector_sbf],
        [mag_field_eci, sun_vector_eci],
        quaternion_prev
    )
    
    satellite.manage_modes()
    satellite.apply_detumbling()
    satellite.apply_pointing()

    satellite.state_vector.next_row()
    ut.basic_state_vector(satellite)
    quaternion_prev = satellite.quaternion.copy() 

satellite.state_vector.to_csv('simulation_state_vector.csv')

print(satellite.state_vector.to_dataframe()) 
plot_orbit(satellite.state_vector.to_dataframe(), setup)
plot_position(satellite.state_vector.to_dataframe())
plot_lla(satellite.state_vector.to_dataframe())
plot_magnetic_field_eci(satellite.state_vector.to_dataframe())
plot_magnetic_field_sbf(satellite.state_vector.to_dataframe()) 
plot_angular_velocity(satellite.state_vector.to_dataframe())
plot_euler_angles(satellite.state_vector.to_dataframe())
plot_pointing_error(satellite.state_vector.to_dataframe())
plot_torque(satellite.state_vector.to_dataframe())
plot_angular_acceleration(satellite.state_vector.to_dataframe())
plot_sun_vector_eci(satellite.state_vector.to_dataframe())
plot_sun_vector_sbf(satellite.state_vector.to_dataframe())


Iteration 0 of 5000
Detumbling stopped (|ω|=0.02 deg/s). Pointing started.
 B-cross: angle=116.35459852948873 deg align dipole =[-0. -0.  0.] damp dipole =[-2.58354681e-03  2.61679634e-03 -3.32495260e-05]
   approach_gate: dθ/dt=0.000  ramp=0.00
 B-cross: angle=114.16288921852156 deg align dipole =[-0.01357574 -0.01597058  0.01103778] damp dipole =[ 0.00897141  0.01050384 -0.00727511]
   approach_gate: dθ/dt=-0.111  ramp=0.08
 B-cross: angle=111.1537754810421 deg align dipole =[0. 0. 0.] damp dipole =[ 0.01123131  0.0140401  -0.00873065]
   approach_gate: dθ/dt=-0.129  ramp=0.17
 B-cross: angle=108.13700449816125 deg align dipole =[-0.04183027 -0.05640335  0.03123053] damp dipole =[ 0.0089537   0.01193162 -0.00663821]
   approach_gate: dθ/dt=-0.113  ramp=0.25
Iteration 100 of 5000
 B-cross: angle=105.08459613859858 deg align dipole =[0. 0. 0.] damp dipole =[ 0.0111367   0.01586616 -0.0078308 ]
   approach_gate: dθ/dt=-0.130  ramp=0.33
 B-cross: angle=102.0319669626605 deg align dipole 

In [ ]:
# TODO
# complete b-cross
# add bang-bang mode for b-dot
# test all b-dot modes
# move all initial parameters to settings (sensors, actuators, satellite, simulation)
# add on-off time for actuators and sensors 
# use better visualizations (interactive, zoom in/out   )
# some optimization
# verify the descriptions and documentation

# OPTIONAL
# add more sensors
# add more actuators
# tests for main parameters


Documentation:

1. Whats in this code

2. Short intro
    what is adcs
    simulating environment
    simulating sensors and actuartors
    basic control

3. Threoretical background
    simlating envireonment (tle, magnetic field, sun position etc)
    reference frames and transformations
    sensors (magnetometer, sun sensor, star tracker, gyroscope)


In [ ]:
import numpy as np
#                      x                 y                   z              x               y
array1 = np.array([-0.0114, -        -0.0114,          0.0114]) #         -0.0114, -        0.0114
array2 = np.array([ 1.92709051e-05, -3.48258106e-05,  1.31687787e-05]) # 1.92709051e-05, -3.48258106e-05

array3 = [
    (array1[1]*array2[2] - array1[2]*array2[1]),
    (array1[2]*array2[0] - array1[0]*array2[2]),
    (array1[0]*array2[1] - array1[1]*array2[0])
]

array3_np = np.cross(array1, array2)    

print(array3)
print(array3_np)
